<a href="https://colab.research.google.com/github/MelihKlc/Deep-Learning/blob/main/Transfer_Learning_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning - Feature Extraction

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem

There are two main benefits :

1. Can leverage an existing neural network architecture proven to work on problems similar to our own.
2. Can leverage a working neural network architecture which has already learned patterns on similar data to our own , then we can adapt those patterns to our own data

#Load and becoming one with the data

In [45]:
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2023-08-05 16:00:57--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.125.128, 142.250.157.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.5’

10_food_classes_10_ 100%[===================>] 160.74M  29.6MB/s    in 6.2s    

2023-08-05 16:01:04 (25.9 MB/s) - ‘10_food_classes_10_percent.zip.5’ saved [168546183/168546183]



In [46]:
import os

# Walk through pizza_steak directory and list number of files
for dirpath , dirnames , filenames , in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}' " )

There are 2 directories and 0 images in '10_food_classes_10_percent' 
There are 10 directories and 0 images in '10_food_classes_10_percent/test' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream' 
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ch

# Burada transfer learningin gücünü görüyoruz. Her test classında bir önceki model gibi 250 image var fakat bu sefer train datamızda her class 75 image içeriyor. Yani biz 750 image la train ettigimiz modelde bile yüksek accuracy alamamıstık fakat transfer learning ile 75 image ile yüksek bir accuracy yakalayabiliyoruz

In [47]:
# Creating data loaders (preprocess the data)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/10_food_classes_10_percent/train"
test_dir =  "/content/10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen =  ImageDataGenerator(rescale = 1./255)

train_data = train_datagen.flow_from_directory(directory = train_dir ,
                                               batch_size = 32 ,
                                               target_size = (224,224) ,
                                               class_mode = "categorical"
                                               )
test_data = test_datagen.flow_from_directory(directory = test_dir ,
                                               batch_size = 32 ,
                                               target_size = (224,224) ,
                                               class_mode = "categorical"
                                               )

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


# Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks:

* Tracking experiments with the tensorflow callback.
* Model checkpoint with the ModelCheckpoint callback.
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callback.

In [48]:
# Create a tensorboard callback (functionized because we need to create a new one for each model)

import datetime  #modelleri yaptığımız tarihleri yazdırmak için. Yani mesela çarşamba bi model yaptın cuma günü yaptın bi de ikisini karşılaştırıyo.

def create_tensorboard_callback(dir_name , experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"Saving Tensorboard log files to : {log_dir}")
  return tensorboard_callback

# Its main functionality is saving a model's training performance metrics to a specified log_dir

# Creating models using tensorflow hub

In the past , we have used tensorflow to create our own models layer by layer from scratch.

Now we are going to do a similar process , except the majority of our model's layers are going to come from tensorflow hub.

We can access pretrained model on : https://tfhub.dev/

# https://paperswithcode.com/

In [49]:
# Lets compare these two models
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [50]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [51]:
# Lets make a create_model() function to create a model from a URL

def create_model(model_url , num_classes = 10) :
  """
  Takes a tensorflow hub URL and create a keras sequential model with it

  Args :
    model_url (str) = A tensorflow hub feature extraction URL
    num_classes (int) =  Number of output neurons in the output layer.
         should be equal to the number of target classes , default 10

    Returns :
     An uncomplicated keras sequential model with model_url as feature extractor layer and Dense output layer
     with num_classes output neurons.
  """
  # Download the pre-trained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url ,
                                           trainable = False ,  #Freeze the already learned patterns
                                           name = "feature_extractor_layer" ,
                                           input_shape = (224,224,3))

  #Create our own model
  model = tf.keras.Sequential(
      [ feature_extractor_layer  ,
       layers.Dense(num_classes , activation = "softmax" , name = "output_layer" )

        ]

  )
  return model


# Creating and testing resnet tensorflow hub feature extraction model

In [52]:
# Create resnet model
resnet_model = create_model(model_url = resnet_url )

In [53]:
#Compile our resnet model
resnet_model.compile(loss = ["categorical_crossentropy"] ,
                     optimizer = tf.keras.optimizers.Adam() ,
                     metrics = ["accuracy"]
                     )


In [54]:
# Fit the model
resnet_model.fit(train_data , epochs = 5 , steps_per_epoch = len(train_data) , validation_data = test_data , validation_steps = len(test_data) ,
                 callbacks = [create_tensorboard_callback(dir_name = "tensorflow_hub" , experiment_name = "resnet50V2")] )

Saving Tensorboard log files to : tensorflow_hub/resnet50V2/20230805-160111
Epoch 1/5
24/24 [==============================] - ETA: 0s - loss: 1.7869 - accuracy: 0.3867

KeyboardInterrupt: ignored